### IPMOF - InterPenetrating Metal Organic Frameworks

#### Initialize necessary information

1. Read FF_Parameters excel file to get force field parameters for atoms
2. Initialize force field selection, cut_off radius and grid size for energy map

#### Read structural information for MOF files in a given directory
1. Read MOF files in ".mol2" format from given directory and create a list
2. Create MOF objects for structure files
3. Initialize structural information for the MOFs

#### Read simulation input parameters
1. Read simulation parameters from input file

#### Calculate energy map
1. Determine packing amount of the MOF
2. Calculate packed coordinates for the base MOF
3. Calculate energy map

#### Start interpenetration
1. Energy map + mobile_mof

In [12]:
python_lib_dir = r'/home/kutay/Documents/git/IPMOF/IPMOF_Python'
# python_lib_dir = r'C:\Kutay\IPMOF\IPMOF_Python'

excel_file_path = r'/home/kutay/Documents/Research/FF_Parameters.xlsx'
# excel_file_path = r'C:\Users\kutay\iPython\IPMOF\FF_Parameters.xlsx'

mol2_dir = r'/home/kutay/Documents/Research/MOFs/IPMOF_Python/mol2'
# mol2_dir = r'C:\Kutay\MOFs\IPMOF_Python'

base_mof_index = 4
mobile_mof_index = 4

input_dir = r'/home/kutay/Documents/git/IPMOF'
sim_par_path = os.path.join(input_dir, 'sim_par.yaml')

#export_dir = r'C:\Kutay\MOFs\IPMOF_Python\export'
export_dir = r'/home/kutay/Documents/Research/MOFs/IPMOF_Python/export'

In [ ]:
# Dump sim par yaml file
import yaml

sim_par_file = open(sim_par_path, 'w')
sim_par_data = {'structure_energy_limit': 3E3,
                'atom_energy_limit': 3E1,
                'rotation_limit': 20,
                'rotation_freedom': 30,
                'summary_percent': 5,
                'cut_off': 12,
                'ext_cut_off': 50
               }

yaml.dump(sim_par_data, sim_par_file)
sim_par_file.close()

In [14]:
import yaml
# Read sim par yaml file
sim_par = yaml.load(open(sim_par_path, 'r'))

In [3]:
import math
import os

# Load interpenetration python libraries
os.chdir(python_lib_dir)
from forcefield import read_ff_parameters
from crystal import *
from energymap import *
from visualize import *
from interpenetration import *
from quaternion import *
%pylab inline

# Read excel file containing force field information
uff = read_ff_parameters(excel_file_path, 'uff')

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### File Input Options
- <b> Built-in read_mol2 </b>
<pre>
uc_size, uc_angle, atom_names, atom_coors = read_mol2(mol2_path)
mof = MOF()
mof.initialize(mof.mol2_path)
</pre> 
- <b> Ase pdb read (cif read gives error) </b>
<pre>
from ase.io import read
mof_atoms = read(mof_dir, format='pdb')
mof_atoms.get_positions()                # Coordinates
mof_atoms.get_chemical_symbols()         # Atom names
mof_atoms.get_cell()                     # Cell vectors? (check)
mof_obj.get_number_of_atoms()            # Num of atoms
mof_obj.get_volume()                     # Unit cell volume

</pre>
- <b> Open babel </b>
<pre>
babel -icif *.cif -opdb *.pdb
</pre>

In [4]:
# Create list of MOFs
mol2_list = get_mof_list(mol2_dir, '.mol2')
print(mol2_list)

['QIGBIR.mol2', 'OFODAP.mol2', 'UNIGEE.mol2', 'ZIF90.mol2', 'SAHYIK.mol2', 'KINFAQ.mol2', 'NUVWIL.mol2']


In [5]:
# Read mol2 files and initialize MOF objects
base_mof = MOF()
base_mof.mol2_path = os.path.join(mol2_dir, mol2_list[base_mof_index])
base_mof.initialize()
base_mof.initialize_ff(uff)
print('Base MOF selected as: ', base_mof.name)

mobile_mof = MOF()
mobile_mof.mol2_path = os.path.join(mol2_dir, mol2_list[mobile_mof_index])
mobile_mof.initialize()
mobile_mof.initialize_ff(uff)
print('Mobile MOF selected as: ', mobile_mof.name)

Base MOF selected as:  SAHYIK
Mobile MOF selected as:  SAHYIK


In [6]:
cut_off = sim_par['cut_off']

base_mof.packing_factor = Packing.factor(base_mof.uc_size, cut_off)
uc_vectors = Packing.uc_vectors(base_mof.uc_size, base_mof.uc_angle)
trans_vec = Packing.translation_vectors(base_mof.packing_factor, uc_vectors)
base_mof.packed_coors = Packing.uc_coors(trans_vec, base_mof.packing_factor, uc_vectors, base_mof.atom_coors)
base_mof.edge_points = Packing.edge_points(uc_vectors)

print('Base MOF unit cell: ', base_mof.uc_size)
print('Packing factor:', base_mof.packing_factor)
print('Num of coor :', len(base_mof.packed_coors)*len(base_mof.packed_coors[0]))

Base MOF unit cell:  [25.669, 25.669, 25.669]
Packing factor: [3, 3, 3]
Num of coor : 11448


In [7]:
atom_list = get_uniq_atom_list([mobile_mof])
print('Calculating emap for', base_mof.name, 'with atoms:', atom_list['atom'])
emap = energy_map(base_mof, atom_list, cut_off, 1)

Calculating emap for SAHYIK with atoms: ['H', 'C', 'O', 'Zn']


In [22]:
emap_max = [emap[-1][0], emap[-1][1], emap[-1][2]]
emap_min = [emap[0][0], emap[0][1], emap[0][2]]

coor = [1.89, 11, 4.78]
emap_index = energy_map_index(coor, emap_max, emap_min)
atom_index = energy_map_atom_index('C', atom_list)

print('Energy map energy at ', emap[emap_index][:3], ' : ', emap[emap_index][atom_index] )
interpolation_energy = trilinear_interpolate(coor, atom_index, emap, emap_max, emap_min)
print('Interpolation energy at ', coor, ' : ', interpolation_energy)

Energy map energy at  [  2.  11.   5.]  :  -562.512156392
Interpolation energy at  [1.89, 11, 4.78]  :  -534.356103682


In [23]:
summary, new_structures = run_interpenetration(sim_par, base_mof, mobile_mof, emap, atom_list)

In [14]:
# Get minimum energy structure by sorting total structure energies
min_energy_structure = sorted(new_structures, key=lambda k: k['energy'])[0]

rotation_info = min_energy_structure['rotation']

In [24]:
def check_extension(sim_par, base_MOF, mobile_MOF, emap, emap_atom_list, rotation_info):
    """
    *** Not Complete ***
    Checks collision between interpenetrating layer and base layer for a determined distance.
    Distance is calculated from given ext_cut_off value which determines the packing amount of the
    interpenetrating layer.
    Each coordinate in the interpenetrating layer is checked for high energy values by applying
    perodic boundary conditions to the coordinate according to energy map of the base layer.
    """
    emap_max = [emap[-1][0], emap[-1][1], emap[-1][2]]
    emap_min = [emap[0][0], emap[0][1], emap[0][2]]
    
    energy_limit = sim_par['atom_energy_limit']
    #energy_limit = 3E20
    ext_cut_off = sim_par['ext_cut_off']
    
    Quat = Quaternion([0, 1, 1, 1])

    packing_factor = Packing.factor(mobile_MOF.uc_size, ext_cut_off)
    uc_vectors = Packing.uc_vectors(mobile_MOF.uc_size, mobile_MOF.uc_angle)
    trans_vec = Packing.translation_vectors(packing_factor, uc_vectors)
    packed_coors = Packing.uc_coors(trans_vec, packing_factor, uc_vectors, mobile_MOF.atom_coors)

    #rotated_packed_coors = rotate_unit_cell(packed_coor, rotation_info)
    x_angle = rotation_info[0]
    y_angle = rotation_info[1]
    z_angle = rotation_info[2]

    collision = False
    for unit_cell in packed_coors:
        
        if not collision:
            
            for coor_index, coor in enumerate(unit_cell):

                if not collision:

                    new_coor = Coor(coor)
                    Q = Quaternion([1, new_coor.x, new_coor.y, new_coor.z])  # Might be a better way to do this
                    Q = Quat.rotation(Q.xyz(), [0, 0, 0], [1, 0, 0], x_angle)
                    Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 1, 0], y_angle)
                    Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 0, 1], z_angle)
                    new_coor = Coor(Q.xyz())

                    pbc_coor = new_coor.pbc(base_MOF.uc_size, base_MOF.uc_angle, base_MOF.frac_ucv)

                    #emap_index = energy_map_index(pbc_coor.xyz(), emap_max, emap_min)
                                              
                    atom_name = mobile_MOF.atom_names[coor_index]
                    atom_index = energy_map_atom_index(atom_name, emap_atom_list)

                    point_energy = trilinear_interpolate(pbc_coor.xyz(), atom_index, emap, emap_max, emap_min)
                    print(point_energy)
                    #energy = emap[emap_index][atom_index]
                    if point_energy < energy_limit:
                        continue
                    else:
                        collision = True
                        break
                else:
                    break
        else:
            break

    return collision

In [ ]:
def save_extension(sim_par, base_MOF, mobile_MOF, emap, emap_atom_list, rotation_info):
    """
    *** Not Complete ***
    Checks collision between interpenetrating layer and base layer for a determined distance.
    Distance is calculated from given ext_cut_off value which determines the packing amount of the
    interpenetrating layer.
    Each coordinate in the interpenetrating layer is checked for high energy values by applying
    perodic boundary conditions to the coordinate according to energy map of the base layer.
    """
    emap_max = [emap[-1][0], emap[-1][1], emap[-1][2]]
    emap_min = [emap[0][0], emap[0][1], emap[0][2]]
    
    #ext_cut_off = sim_par['ext_cut_off']
    ext_cut_off = 20
    
    Quat = Quaternion([0, 1, 1, 1])

    packing_factor = Packing.factor(mobile_MOF.uc_size, ext_cut_off)
    uc_vectors = Packing.uc_vectors(mobile_MOF.uc_size, mobile_MOF.uc_angle)
    trans_vec = Packing.translation_vectors(packing_factor, uc_vectors)
    packed_coors = Packing.uc_coors(trans_vec, packing_factor, uc_vectors, mobile_MOF.atom_coors)

    x_angle = rotation_info[0]
    y_angle = rotation_info[1]
    z_angle = rotation_info[2]
    
    extended_coors = []
    extended_names = []

    for unit_cell in packed_coors:

        for coor_index, coor in enumerate(unit_cell):

            new_coor = Coor(coor)
            Q = Quaternion([1, new_coor.x, new_coor.y, new_coor.z])  # Might be a better way to do this
            Q = Quat.rotation(Q.xyz(), [0, 0, 0], [1, 0, 0], x_angle)
            Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 1, 0], y_angle)
            Q = Quat.rotation(Q.xyz(), [0, 0, 0], [0, 0, 1], z_angle)
            new_coor = Coor(Q.xyz())

            atom_name = mobile_MOF.atom_names[coor_index]
            
            extended_names.append(atom_name)
            extended_coors.append(new_coor.xyz())


    return extended_names, extended_coors

In [33]:
ext_names, ext_coors = save_extension(sim_par, base_mof, mobile_mof, emap, atom_list, rotation_info)

export_xyz(ext_coors, ext_names, 'mobile_ext', export_dir)

In [ ]:
collision = check_extension(sim_par, base_mof, mobile_mof, emap, atom_list, rotation_info)

In [17]:
def join_structures(base_mof, new_structure):
    joined_atom_names = []
    joined_atom_coors = []
    for atom, coor in zip(new_structure['atom_names'], new_structure['atom_coors']):
        joined_atom_names.append(atom)
        joined_atom_coors.append(coor)
        
    for atom, coor in zip(base_mof.atom_names, base_mof.atom_coors):
        joined_atom_names.append(atom)
        joined_atom_coors.append(coor)
        
    return joined_atom_names, joined_atom_coors

# Define file name and export directory
file_name = base_mof.name + '_' + mobile_mof.name
export_dir = r'C:\Kutay\MOFs\IPMOF_Python\export'
#export_dir = r'/home/kutay/Documents/Research/MOFs/IPMOF_Python/export'

# Join interpenetrating structures
joined_names, joined_coors = join_structures(base_mof, min_energy_structure)

# Export coordinates in .xyz format
export_xyz(joined_coors, joined_names, file_name, export_dir)

In [ ]:
packing_factor = Packing.factor(mobile_mof.uc_size, 50)
uc_vectors = Packing.uc_vectors(mobile_mof.uc_size, mobile_mof.uc_angle)
trans_vec = Packing.translation_vectors(packing_factor, uc_vectors)
packed_coors = Packing.uc_coors(trans_vec, packing_factor, uc_vectors, mobile_mof.atom_coors)

In [112]:
packed_mof_path = os.path.join(export_dir, 'packed.xyz')
packed_mof_file = open(packed_mof_path, 'w')
packed_mof_file.write(str(len(packed_coors)*len(packed_coors[0])) + '\n')
packed_mof_file.write('Packed \n')

for unit_cell in packed_coors:
    for coor_index, coor in enumerate(unit_cell):
        atom_name = mobile_mof.atom_names[coor_index]
        packed_mof_file.write(atom_name + ' ' + str(coor[0]) + ' ' + str(coor[1]) + ' ' + str(coor[2]) + '\n')
        
packed_mof_file.close()